In [ ]:
!pip install mowl-borg

# mOWL: Python library for machine learning with ontologies

Ontologies are being used across different machine learning models
since they can provide structured and axiomatic background knowldege regarding a particular domain of interest.


<img src="figs/mowl_intro/ont_repr2.jpg" width="600" alt="Ontology with axioms and annotations." title="Ontology with axioms and annotations."/>

In mOWL, we have implemented a set of tools and functionalities to ease the integration of methods that use ontologies. The features that mOWL contains are:

* Ontology Management
* Ontology Transformation
* Embedding Generation
* Embedding Post-processing

## Ontology Management

This includes ontology normalization and reasoning. To achieve this,
we interface with the OWL API.

<img src="figs/mowl_intro/reasoning.jpg" width="400" alt="Ontology reasoning." title="Ontology reasoning."/>


## Ontology Transformation
This includes methods for extracting ontology information into graphs, text corpora or process the axioms directly.

<img src="figs/mowl_intro/transformation.jpg" width="400" alt="Ontology transformation." title="Ontology transformation."/>

## Embedding Generation

Each ontology transformation result is usable to generate vector representations (embeddings) of entities. For graph projections we can use random walk methods (implemented in mOWL) or interface with the knowledge graph embedding library PyKEEN. Transformation into text allows to use natural language processing methods.

<img src="figs/mowl_intro/embeddings.jpg" width="600" alt="Embeddings generation." title="Embeddings generation."/>

## Embedding post-processing

This includes methods to use the embeddings previously generated. We can directly evaluate them using a set of metrics or use them to train a machine learning model.

# Getting started

To get started, you can install mOWL using pip:

mOWL interfaces the OWL API. For this, we need to interface with the Java Virtual Machine (JVM):

In [1]:
import mowl
mowl.init_jvm("10g")

In [ ]:
import os
from java.util import HashSet
from mowl.owlapi import OWLAPIAdapter
from org.semanticweb.owlapi.model import IRI

# Let's create our first ontology

In [ ]:
adapter = OWLAPIAdapter()
ontology = adapter.create_ontology("http://mowl/family")

## Concept names

In [ ]:
male = adapter.create_class("http://Male")
female = adapter.create_class("http://Female")
parent = adapter.create_class("http://Parent")
person = adapter.create_class("http://Person")
mother = adapter.create_class("http://Mother")
father = adapter.create_class("http://Father")
sibling = adapter.create_class("http://Sibling")
brother = adapter.create_class("http://Brother")
sister = adapter.create_class("http://Sister")
son = adapter.create_class("http://Son")
daughter = adapter.create_class("http://Daughter")

## Role names

In [ ]:
has_child = adapter.create_object_property("http://hasChild")
has_parent = adapter.create_object_property("http://hasParent")

## Individual names

In [ ]:
John = adapter.create_individual("http://John")
Jane = adapter.create_individual("http://Jane")
Robert = adapter.create_individual("http://Robert")
Melissa = adapter.create_individual("http://Melissa")

## Axioms

Let's create some axioms of the form $A \sqsubseteq B$.

In [ ]:
axioms = HashSet()
axioms.add(adapter.create_subclass_of(male, person))
axioms.add(adapter.create_subclass_of(female, person))
axioms.add(adapter.create_subclass_of(parent, person))
axioms.add(adapter.create_subclass_of(mother, female))
axioms.add(adapter.create_subclass_of(father, male))

Now, let's create some axioms of the form $A \sqcap B \sqsubseteq C$.

In [ ]:
parent_and_male = adapter.create_object_intersection_of(parent, male)
axioms.add(adapter.create_subclass_of(parent_and_male, father))
parent_and_female = adapter.create_object_intersection_of(parent, female)
axioms.add(adapter.create_subclass_of(parent_and_female, mother))

Now some axioms of the form $A \sqcup B \equiv C$.

In [ ]:
male_or_female = adapter.create_object_union_of(male, female)
axioms.add(adapter.create_equivalent_classes(male_or_female, person))

One axiom of the form $\neg A \equiv  B$.

In [ ]:
not_male = adapter.create_complement_of(male)
axioms.add(adapter.create_equivalent_classes(not_male, female))

One axiom of the form $A \sqsubseteq \exists R.B$.

In [ ]:
has_child_person = adapter.create_object_some_values_from(has_child, person)
axioms.add(adapter.create_subclass_of(parent, has_child_person))

And finally, some assertion axioms of the form $C(a)$ and $R(a,b)$.

In [ ]:
axioms.add(adapter.create_class_assertion(father, John))
axioms.add(adapter.create_class_assertion(mother, Jane))
axioms.add(adapter.create_class_assertion(male, Robert))
axioms.add(adapter.create_class_assertion(female, Melissa))
axioms.add(adapter.create_object_property_assertion(has_child, John, Robert))
axioms.add(adapter.create_object_property_assertion(has_child, Jane, Robert))
axioms.add(adapter.create_object_property_assertion(has_child, John, Melissa))
axioms.add(adapter.create_object_property_assertion(has_child, Jane, Melissa))
adapter.owl_manager.addAxioms(ontology, axioms)

In [ ]:
ont_file = os.path.abspath(f'family.owl')

In [ ]:
adapter.owl_manager.saveOntology(ontology, IRI.create('file://'+ont_file))